In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../src/")
sys.path.insert(0, "../../../src/")
sys.path.insert(0, "../../../projects/skimpy_bench/src/")
sys.path.insert(0, "../../../projects/skimpy_blox/")

In [3]:
import random
import numpy as np
import skimpy
import skimpy_3d
from skimpy_blox import minecraft, colors
from skimpy import functional as F
import ipywebrtc

SHOW_RUNS_MODE = True

In [4]:
def show_runs(t, runs_value=2):
    q = t.reshape(len(t))
    ret = skimpy.Tensor(len(t), val=q.to(bool).to(int)._tensor.array())
    run_points = runs_value * (
        (
            (q[:(len(q) - 1)] == q[1:])
        )
        & (q[1:] != 0)
    ).to(int)

    ret[1:] = ret[1:].max(run_points)
    ret[0:(len(ret) - 1)] = ret[0:(len(ret) - 1)].max(runs_value * (ret[1:] == 2).to(int))
    return ret.reshape(t.shape)
    

In [5]:
%%time
# level = minecraft.SkimpyMinecraftLevel.load("../data/sample_minecraft_world.pickle.gz")
level = minecraft.SkimpyMinecraftLevel.load("../data/Hogwarts1.0.pickle.gz")

CPU times: user 598 ms, sys: 61.7 ms, total: 660 ms
Wall time: 660 ms


In [6]:
%%time
t = level.megatensor()
# t = level.chunk_list[0].tensor
if SHOW_RUNS_MODE:
    t = show_runs(t)

CPU times: user 25.5 s, sys: 1.99 s, total: 27.5 s
Wall time: 3.14 s


In [7]:
for i in range(20):
    print(f"{i}=>{F.sum((t == i).to(int))}")

0=>669083423
1=>461406
2=>12291715
3=>0
4=>0
5=>0
6=>0
7=>0
8=>0
9=>0
10=>0
11=>0
12=>0
13=>0
14=>0
15=>0
16=>0
17=>0
18=>0
19=>0


In [8]:
config = skimpy_3d.VoxelConfig()
config[0] = skimpy_3d.EmptyVoxel()
if SHOW_RUNS_MODE:
    config[1] = skimpy_3d.ColorVoxel(255, 255, 255)
    config[2] = skimpy_3d.ColorVoxel(255, 105, 180)
else:    
    for i in range(1, 256):
        r, g, b = colors.color_for_id(i)
        config[i] = skimpy_3d.ColorVoxel(r, g, b)

In [9]:
%%time
mesh = skimpy_3d.generate_mesh(config, t._tensor)

CPU times: user 6.28 s, sys: 581 ms, total: 6.87 s
Wall time: 1.73 s


342291312

In [27]:
(len(mesh.triangles) * 4  + len(mesh.positions) * 4) / (level.num_runs() * 2 * 4) 

29.843810116358068

In [10]:
from pythreejs import *
import numpy as np
from IPython.display import display

In [11]:
%%time 

geometry = BufferGeometry(
    attributes={
        "position": BufferAttribute(
            np.array(mesh.positions, dtype='float32').reshape(-1, 3),
            normalized=False,
        ),
        "normal": BufferAttribute(
            np.array(mesh.normals, dtype='float32').reshape(-1, 3),
            normalized=False,
        ),
        "color": BufferAttribute(
            np.array(mesh.colors, dtype='uint8').reshape(-1, 4)
        ),
        "index": BufferAttribute(
            np.array(mesh.triangles, dtype='uint32'),
            normalized=False,
        ),
    },
)

CPU times: user 15.1 s, sys: 1.81 s, total: 16.9 s
Wall time: 16.5 s


In [12]:
js_mesh = Mesh(
    geometry=geometry,
    material=MeshLambertMaterial(vertexColors="VertexColors"),
    position=[0, 0, 0],
)

In [13]:
"""
camera_pos = level.xyz_to_skimpy_col(level.dense_dimensions())
camera = PerspectiveCamera(position=camera_pos, lookAt=(0,0,0), fov=20)
up = [0, 0, 0]
up[level.column_order[1]] = 1
camera.up = tuple(up)
"""

'\ncamera_pos = level.xyz_to_skimpy_col(level.dense_dimensions())\ncamera = PerspectiveCamera(position=camera_pos, lookAt=(0,0,0), fov=20)\nup = [0, 0, 0]\nup[level.column_order[1]] = 1\ncamera.up = tuple(up)\n'

In [14]:
light_position = level.xyz_to_skimpy_col(tuple(e / 2 for e in level.dense_dimensions()))
point_light = PointLight(color='#ffffff', position=light_position)
global_light = AmbientLight(color='#333333')

#normals_helper = VertexNormalsHelper(js_mesh, size=0.5, color="#ffcc00")

scene = Scene(children=[js_mesh, camera, point_light, global_light])

NameError: name 'camera' is not defined

In [ ]:
camera.position

In [ ]:
camera.lookAt

In [ ]:
stream = ipywebrtc.WidgetStream(widget=renderer, max_fps=30)

In [ ]:
recorder = ipywebrtc.ImageRecorder(filename='snapshot', format='png', stream=stream)

In [ ]:
recorder.save("/home/tdimson/Downloads/render.png")

In [ ]:
renderer = Renderer(
    camera=camera, 
    background="#b0b0b0", 
    background_opacity=1,
    scene=scene,
    controls=[OrbitControls(controlling=camera)],
    width=984,
    height=984,
)

display(renderer)